In [33]:
import numpy as np
import os
import time
import argparse
import random

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input,
    Concatenate,
    Flatten,
    BatchNormalization,
    Activation,
    GlobalAveragePooling1D,
)
from tensorflow.keras.optimizers import Adam
from qkeras import QActivation, QDense, QConv1D, QConv2D, quantized_bits
from qkeras.autoqkeras.utils import print_qmodel_summary

from sklearn.metrics import accuracy_score
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_wrapper


from node_edge_projection import NodeEdgeProjection

In [2]:
jetConstituent = np.load("data/jetConstituent_150_16f.npy")
target = np.load("data/jetConstituent_target_150_16f.npy")

In [3]:
# Restrict the number of constituents to a maximum of NMAX
nmax = 30
jetConstituent = jetConstituent[:, 0:nmax, :]

In [4]:
# The dataset is N_jets x N_constituents x N_features
njet = jetConstituent.shape[0]
nconstit = jetConstituent.shape[1]
nfeat = jetConstituent.shape[2]


print("Number of jets =", njet)
print("Number of constituents =", nconstit)
print("Number of features =", nfeat)


Number of jets = 260000
Number of constituents = 30
Number of features = 16


In [5]:

# Shuffles jet constituents
print("Before --->> jetConstituent[0,0:4,0] = ", jetConstituent[0, 0:4, 0])
for i in range(jetConstituent.shape[0]):
    jetConstituent[i] = jetConstituent[i, np.random.permutation(nconstit), :]
print("After --->> jetConstituent[0,0:4,0] = ", jetConstituent[0, 0:4, 0])


from sklearn.model_selection import train_test_split

X = jetConstituent
Y = target
del jetConstituent, target

X_train_val, X_test, Y_train_val, Y_test = train_test_split(
    X, Y, test_size=0.33, random_state=7
)

print(X_train_val.shape, X_test.shape, Y_train_val.shape, Y_test.shape)

Before --->> jetConstituent[0,0:4,0] =  [-177.97363281 -139.30709839 -122.01608276 -112.20430756]
After --->> jetConstituent[0,0:4,0] =  [-82.26656342 -11.31220341 -36.57773209  -9.24982643]
(174200, 30, 16) (85800, 30, 16) (174200, 5) (85800, 5)


In [6]:
print(
    "number of G jets for training/validation: %i"
    % np.sum(np.argmax(Y_train_val, axis=1) == 0)
)
print(
    "number of Q jets for training/validation: %i"
    % np.sum(np.argmax(Y_train_val, axis=1) == 1)
)
print(
    "number of W jets for training/validation: %i"
    % np.sum(np.argmax(Y_train_val, axis=1) == 2)
)
print(
    "number of Z jets for training/validation: %i"
    % np.sum(np.argmax(Y_train_val, axis=1) == 3)
)
print(
    "number of T jets for training/validation: %i"
    % np.sum(np.argmax(Y_train_val, axis=1) == 4)
)


print("number of G jets for testing: %i" % np.sum(np.argmax(Y_test, axis=1) == 0))
print("number of Q jets for testing: %i" % np.sum(np.argmax(Y_test, axis=1) == 1))
print("number of W jets for testing: %i" % np.sum(np.argmax(Y_test, axis=1) == 2))
print("number of Z jets for testing: %i" % np.sum(np.argmax(Y_test, axis=1) == 3))
print("number of T jets for testing: %i" % np.sum(np.argmax(Y_test, axis=1) == 4))


number of G jets for training/validation: 35213
number of Q jets for training/validation: 33694
number of W jets for training/validation: 35083
number of Z jets for training/validation: 35023
number of T jets for training/validation: 35187
number of G jets for testing: 17191
number of Q jets for testing: 16774
number of W jets for testing: 17152
number of Z jets for testing: 17275
number of T jets for testing: 17408


In [7]:
# baseline keras model

njet = X_train_val.shape[0]
nconstit = X_train_val.shape[1]
ntargets = Y_train_val.shape[1]
nfeat = X_train_val.shape[2]


print("#jets = ", njet)
print("#constituents = ", nconstit)
print("#targets = ", ntargets)
print("#features = ", nfeat)

#jets =  174200
#constituents =  30
#targets =  5
#features =  16


In [34]:
De = 8  # size of latent edges features representations
Do = 6  # size of latent nodes features representations
scale_e = 2  # multiplicative factor for # hidden neurons in Edges MLP
scale_n = 2  # multiplicative factor for # hidden neurons in Nodes MLP
scale_g = float(288/(Do))/(nmax*Do)
NL = 0

# Interaction Network model parameters
N = nconstit
P = nfeat
Nr = N * (N - 1)  # number of relations ( edges )
Dr = 0
Dx = 0

# Quantized bits
nbits = 13
integ = 2

# Set QKeras quantizer and activation
if nbits == 1:
    qbits = "binary(alpha=1)"
elif nbits == 2:
    qbits = "ternary(alpha=1)"
else:
    qbits = "quantized_bits({},{},0,alpha=1)".format(nbits, integ)

qact = "quantized_relu({},{},0)".format(nbits, integ)

# Print
print("Training with max # of contituents = ", nconstit)
print("Number of node features = ", nfeat)
print("Quantization with nbits=", nbits)
print("Quantization of integer part=", integ)

print(qbits)

Training with max # of contituents =  30
Number of node features =  16
Quantization with nbits= 13
Quantization of integer part= 2
quantized_bits(13,2,0,alpha=1)


In [35]:
# Input shape
inp = Input(shape=(nconstit, nfeat), name="in_layer")

# Batch normalize the inputs
x = BatchNormalization(name="batchnorm")(inp)

# Project to edges
ORr = NodeEdgeProjection(name="proj_1", receiving=True, node_to_edge=True)(x)
ORs = NodeEdgeProjection(name="proj_2", receiving=False, node_to_edge=True)(x)

inp_e = Concatenate(axis=-1)(
    [ORr, ORs]
)  # Concatenates Or and Os  ( no relations features Ra matrix )

# Edges MLP ( takes as inputs nodes features of a fully conected graph edges )

# Define the Edges MLP layers
nhidden_e = int((2 * P + Dr) * scale_e)
if NL == 2:
    h = QConv1D(
        nhidden_e,
        kernel_size=1,
        kernel_quantizer=qbits,
        bias_quantizer=qbits,
        name="conv1D_e1",
    )(inp_e)
    h = QActivation(qact, name="qrelu_e1")(h)
    h = QConv1D(
        int(nhidden_e / 2),
        kernel_size=1,
        kernel_quantizer=qbits,
        bias_quantizer=qbits,
        name="conv1D_e2",
    )(h)
    h = QActivation(qact, name="qrelu_e2")(h)
    h = QConv1D(
        De,
        kernel_size=1,
        kernel_quantizer=qbits,
        bias_quantizer=qbits,
        name="conv1D_e3",
    )(h)
if NL == 1:
    h = QConv1D(
        nhidden_e,
        kernel_size=1,
        kernel_quantizer=qbits,
        bias_quantizer=qbits,
        name="conv1D_e1",
    )(inp_e)
    h = QActivation(qact, name="qrelu_e1")(h)
    h = QConv1D(
        De,
        kernel_size=1,
        kernel_quantizer=qbits,
        bias_quantizer=qbits,
        name="conv1D_e3",
    )(h)
elif NL == 0:
    h = QConv1D(
        De,
        kernel_size=1,
        kernel_quantizer=qbits,
        bias_quantizer=qbits,
        name="conv1D_e3",
    )(inp_e)

out_e = QActivation(qact, name="qrelu_e3")(h)

# Project to nodes
out_e = NodeEdgeProjection(name="proj_3", receiving=True, node_to_edge=False)(out_e)

# Nodes MLP ( takes as inputs node features and embeding from edges MLP )

# Concatenate input Node features and Edges MLP output for the Nodes MLP input
inp_n = Concatenate(axis=-1)(
    [x, out_e]
)  #  Original IN was C = tf.concat([N,x,E], axis=1)

# Define the Nodes MLP layers
nhidden_n = int((P + Dx + De) * scale_n)  # number of neurons in Nodes MLP hidden layer
h = QConv1D(
    nhidden_n,
    kernel_size=1,
    kernel_quantizer=qbits,
    bias_quantizer=qbits,
    name="conv1D_n1",
)(inp_n)
h = QActivation(qact, name="qrelu_n1")(h)
h = QConv1D(
    int(nhidden_n),
    kernel_size=1,
    kernel_quantizer=qbits,
    bias_quantizer=qbits,
    name="conv1D_n2",
)(h)
h = QActivation(qact, name="qrelu_n2")(h)
h = QConv1D(
    Do, kernel_size=1, kernel_quantizer=qbits, bias_quantizer=qbits, name="conv1D_n3"
)(h)
out_n = QActivation(qact, name="qrelu_n3")(h)

#  Graph classification MLP

# Flatten input for the Graph classifier MLP
# inp_g = Flatten()(out_n)

# Invariant operation
out_n = QActivation(quantized_bits(bits=12, integer=3, symmetric=0, keep_negative=1))(out_n)
inp_g = GlobalAveragePooling1D(name="pi_gp")(out_n)


# Define Graph classifier MLP  layers
#nhidden_g = int((Do * N) * scale_g)  # Number of nodes in graph MLP hidden layer
nhidden_g = int((Do * N) * scale_g)  # Number of nodes in graph MLP hidden layer

h = QDense(nhidden_g, kernel_quantizer=qbits, bias_quantizer=qbits, name="dense_g1")(inp_g)
h = QActivation(qact, name="qrelu_g1")(h)

h = QDense(nhidden_g, kernel_quantizer=qbits, bias_quantizer=qbits, name="dense_g2")(h)
h = QActivation(qact, name="qrelu_g2")(h)

h = QDense(ntargets, kernel_quantizer=qbits, bias_quantizer=qbits, name="dense_out")(h)
out = Activation("softmax", name="softmax_out")(h)

# create the model
model = Model(inputs=inp, outputs=out)

In [36]:
# Define the optimizer ( minimization algorithm )
optim = Adam(learning_rate=0.0005)

p_en = 1
batch = 512
p_rate = 0.5
if p_en:
    import tensorflow_model_optimization as tfmot
    from tensorflow_model_optimization.sparsity import keras as sparsity
    from tensorflow_model_optimization.python.core.sparsity.keras import pruning_callbacks

    NSTEPS = int (( int(len(X_train_val) * 0.3))/batch)

    def pruneFunction(layer):
        pruning_params = {
            'pruning_schedule': sparsity.PolynomialDecay(
                initial_sparsity=0.0, final_sparsity=p_rate, begin_step=NSTEPS * 2, end_step=NSTEPS * 10, frequency=NSTEPS
            )
        }
        #pruning_params_mlp_e = {
        #    'pruning_schedule': sparsity.PolynomialDecay(
        #        initial_sparsity=0.0, final_sparsity=0.5, begin_step=NSTEPS * 2, end_step=NSTEPS * 10, frequency=NSTEPS
        #    )
        #}
        if isinstance(layer, tf.keras.layers.Conv1D): 
            return tfmot.sparsity.keras.prune_low_magnitude(layer, **pruning_params)
        #if isinstance(layer, tf.keras.layers.Conv1D) and layer.name != 'conv1D_e3':   
        #    return tfmot.sparsity.keras.prune_low_magnitude(layer, **pruning_params)
        #if isinstance(layer, tf.keras.layers.Conv1D) and layer.name == 'conv1D_e3':
        #    return tfmot.sparsity.keras.prune_low_magnitude(layer, **pruning_params_mlp_e)
        if isinstance(layer, tf.keras.layers.Dense) and layer.name != 'dense_out': # exclude output_dense
            return tfmot.sparsity.keras.prune_low_magnitude(layer, **pruning_params)
        return layer

    print_qmodel_summary(model)
    model = tf.keras.models.clone_model(model, clone_function=pruneFunction)

    model.compile(
        optimizer=optim, loss="categorical_crossentropy", metrics=["categorical_accuracy"]
    )
    
    pr = pruning_callbacks.UpdatePruningStep() 
else:

    # Compile the Model
    model.compile(
        optimizer=optim, loss="categorical_crossentropy", metrics=["categorical_accuracy"]
    )

# Model Summary
model.summary()

batchnorm            is normal keras bn layer
conv1D_e3            f=8 quantized_bits(13,2,0,alpha=1) quantized_bits(13,2,0,alpha=1) 
qrelu_e3             quantized_relu(13,2,0)
conv1D_n1            f=48 quantized_bits(13,2,0,alpha=1) quantized_bits(13,2,0,alpha=1) 
qrelu_n1             quantized_relu(13,2,0)
conv1D_n2            f=48 quantized_bits(13,2,0,alpha=1) quantized_bits(13,2,0,alpha=1) 
qrelu_n2             quantized_relu(13,2,0)
conv1D_n3            f=6 quantized_bits(13,2,0,alpha=1) quantized_bits(13,2,0,alpha=1) 
qrelu_n3             quantized_relu(13,2,0)
q_activation_6       quantized_bits(12,3,0)
dense_g1             u=48 quantized_bits(13,2,0,alpha=1) quantized_bits(13,2,0,alpha=1) 
qrelu_g1             quantized_relu(13,2,0)
dense_g2             u=48 quantized_bits(13,2,0,alpha=1) quantized_bits(13,2,0,alpha=1) 
qrelu_g2             quantized_relu(13,2,0)
dense_g3             u=5 quantized_bits(13,2,0,alpha=1) quantized_bits(13,2,0,alpha=1) 

Model: "model_6"
________

In [32]:
outputdir = "prj/nconst{}_nbits{}_De{}_Do{}_NL{}_SE{}_SN{}_{}".format(
    nmax,
    nbits,
    De,
    Do,
    NL,
    scale_e,
    scale_n,
    time.strftime("%Y%m%d-%H%M%S"),
)

print("output dir: ", outputdir)
os.makedirs(outputdir)

output dir:  prj/nconst30_nbits13_De8_Do6_NL0_SE2_SN2_20240912-200832
